In [ ]:


def format_insert(tname, thedic):
    for i,j in thedic.items(): 
        if isinstance(j, list): 
            thedic[i]= f"ARRAY {j}"
    keystr = ",".join([f"{i}" for i,j in thedic.items()]); 
    valstr = ",".join([f"{j}" for i,j in thedic.items()]); 
    
    finalstr = f"""INSERT INTO {tname} ({keystr}) VALUES ({valstr});"""
    return finalstr

def read_data(result): 
    retdata = pd.DataFrame(result, columns=["f_id", "feature", "point_cloud", "density", "gen_date"])
    return retdata

def point_density(feature):
    feature = str(feature); 
    density = len(feature.replace("0", ""))/len(feature)
    return density

x = np.random.normal(20, 1.5, 100)
y = np.random.normal(10, 8.6, 100)
z = np.random.normal(35, 5.4, 100)

point_stack = np.vstack((x,y,z)).T.tolist()
feature_64 = hash().__str__()[:4]*16
thedic = {
    "feature" : feature_64, 
    "density" : point_density(feature_64), 
    "point_cloud": point_stack, 
}

teststr = format_insert("mass", thedic)



conn = pg.connect("dbname='testfeatures' user='yang' host='localhost' password='testyang'")

cur = conn.cursor()

cur.execute(teststr)
cur.execute("SELECT * from mass;")

results = cur.fetchall()
print(results)

conn.commit()
cur.close()
conn.close()


### Data Structure 
<pre>
 f_id        | integer               |           | not null | nextval('mass_f_id_seq'::regclass)
 feature     | character varying(64) |           |          | 
 point_cloud | numeric[]             |           |          | 
 density     | numeric(4,3)          |           |          | 
 gen_date    | date
</pre>

In [ ]:
import json, time
import psycopg2 as pg
import pandas as pd 
import numpy as np 

class Connection:
    def __init__(self):
        self.connect = False; 
        self.__curs = []; 
        self.__connect_info = {
            "user": "user", 
            "dbname": "database", 
            "host": "localhost", 
            "password": "",
        }
        
    def __insert_str_dict(self, tname, thedic):
        for i,j in thedic.items(): 
            if isinstance(j, list): 
                thedic[i]= f"ARRAY {j}"
        keystr = ",".join([f"{i}" for i,j in thedic.items()]); 
        valstr = ",".join([f"{j}" for i,j in thedic.items()]); 
        finalstr = f"""INSERT INTO {tname} ({keystr}) VALUES ({valstr});"""
        return finalstr
    
    def __connect_str(self): 
        strlst = [f"{i}='{j}'" for i,j in self.__connect_info.items() if len(j) > 0]; 
        finalstr = " ".join(strlst); 
        return finalstr; 
    
    def read_conf(self, conffile): 
        with open(conffile) as file1: 
            confs = json.load(file1); 
        for i in self.__connect_info.keys(): 
            if i in confs.keys(): 
                self.__connect_info[i] = confs[i]; 
    
    def connectit(self):
        connectstr =self.__connect_str(); 
        try: 
            print(f"Successfully connect to the database", self.__connect_info["dbname"], "as user", self.__connect_info["user"])
            self.connect = pg.connect(connectstr); 
        except: 
            print("Fatal: Cannot connect to the database based on the given info");
            print(f"Please carefully check the following connection setting: \n{connectstr}"); 
        
    def insert(self, tname, thedic, clean=True): 
        insertstr = self.__insert_str_dict(tname, thedic); 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(insertstr);
            self.connect.commit(); 
            if clean != True: 
                self.__curs.append(cur); 
            else: 
                cur.close(); 
        except: 
            print("Failed to execute the insertion. "); 
            cur.close(); 
            
    def doit(self, cmd): 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(cmd); 
            self.connect.commit(); 
            cur.close(); 
        except: 
            print(f"Failed to execute the command. {cmd}"); 
            cur.close(); 
        return results
    
    def select(self, selectstr, clean=True): 
        cur = self.connect.cursor(); 
        try: 
            cur.execute(selectstr); 
            results = cur.fetchall(); 
            if clean != True: 
                self.__curs.append(cur); 
            else: 
                cur.close(); 
        except: 
            print("Failed to execute the selection. "); 
            cur.close(); 
        return results
    
    def clear(self):
        self.connect.close()
        if len(self.__curs) > 0:
            print("cleaning cursors")
            [i.close() for i in self.__curs]; 
        
    
# cur.execute("SELECT * from mass;"); 

conffile = "/Users/yangzhang/Dropbox/Documents/BetaPose/myconfig.json"
agent = Connection();
agent.read_conf(conffile); 
agent.connectit(); 

domodify = False

if domodify: 
    print("doing insertion")
    point_stack = np.vstack((x,y,z)).T.tolist()
    feature_64 = time.perf_counter().__str__()[-8:]*8
    thedic = {
        "feature" : feature_64, 
        "density" : point_density(feature_64), 
        "point_cloud": point_stack, 
    }
    
    agent.update("mass", thedic)
agent.doit("delete from mass where f_id<8")
ret = agent.select("SELECT * from mass;"); 
ret2 = agent.select("SELECT * from mass where density <=0.4;"); 
    
agent.clear()
# print(len(ret2))
# print(ret2)

In [ ]:

x = np.random.normal(20, 1.5, 100)
y = np.random.normal(10, 8.6, 100)
z = np.random.normal(35, 5.4, 100)

thestack = np.vstack((x,y,z)).T
print(thestack.shape)
print()
# print(thestack.T)
thestack.tolist()


In [ ]:
read_data(ret)

In [ ]:
import json

connect_info = {
    "user": "user", 
    "dbname": "database", 
    "host": "localhost", 
}
print(json.dumps(connect_info, indent=2     ))

In [ ]:
import pytraj as pt 